In [11]:
import pandas as pd
import numpy as np

from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.cross_validation import StratifiedKFold
from xgboost import XGBClassifier

from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
# set random seed
np.random.seed(1294)

In [3]:
train = pd.read_csv('../data/train.csv', index_col='ID')
test = pd.read_csv('../data/test.csv', index_col='ID')

In [4]:
# get features with zero standard deviation
def get_constant_features(df):
    columns = df.columns
    return [col for col in columns if df[col].std() == 0.0]

constant_features = get_constant_features(train)

# get features which are identical to other features
def get_identical_features(df):
    columns = df.columns
    identical_feat = []
    
    for i in range(len(columns)):
        for j in range(i+1, len(columns)):
            if (df[columns[i]] == df[columns[j]]).all():
                identical_feat.append(columns[i])
    
    return identical_feat

identical_feat = get_identical_features(train)

In [6]:
def get_features_to_remove(constant_features, identical_features):
    features_to_remove = []
    
    for feat in constant_features:
        features_to_remove.append(feat)
    
    for feat in identical_features:
        features_to_remove.append(feat)
    
    return features_to_remove

remove_features = get_features_to_remove(constant_features, identical_feat)
remove_features.append('TARGET')

In [7]:
reduced_features = train.columns.drop(remove_features)
X = train[reduced_features]
y = train.TARGET

test = test[reduced_features]

In [8]:
# replace -999999.000000 with 2 ( most common value )
X = X.replace(-999999.000000, 2)
test = test.replace(-999999.000000, 2)

## Feature Engineering

In [9]:
# number of zeros in each row as feature
X.loc[:, 'num_zeros'] = (X == 0).astype(int).sum(axis=1)
test.loc[:, 'num_zeros'] = (test == 0).astype(int).sum(axis=1)

## Train and Test Split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, stratify=y, random_state=44)

### Modelling

In [79]:
# does pca increase the overall performance for different classfiers ?
pca = PCA(n_components=20, whiten=True).fit(X_train)
print 'Explained variance %f ' %np.sum(pca.explained_variance_ratio_)

Explained variance 1.000000 


In [80]:
pca = PCA(n_components=20, whiten=True)
clf = XGBClassifier(n_estimators=50, learning_rate=0.2, seed=4242, subsample=0.9, colsample_bytree=0.6)
model = Pipeline([('pca', pca), ('clf', clf)])

model.fit(X_train, X_test)

ValueError: bad input shape (38010, 307)

## Stacking

In [69]:
## Creates blending set for training and test and list of classifiers

def get_blending_sets(X_train, y_train, X_test, n_folds=5,):
    n_trees = 100
    n_folds = n_folds

    # Our level 0 classifiers
    clfs = [
        RandomForestClassifier(n_estimators = n_trees, criterion = 'gini', n_jobs=-1),
        ExtraTreesClassifier(n_estimators = n_trees * 2, criterion = 'gini', n_jobs=-1),
        XGBClassifier(learning_rate=0.01, seed=1234, n_estimators=n_trees * 2, max_depth=6, min_child_weight=5, colsample_bytree=0.7, subsample=0.9),
        XGBClassifier(learning_rate=0.1, seed=1234, n_estimators=n_trees, max_depth=3, min_child_weight=5, colsample_bytree=0.7, subsample=0.9),
        XGBClassifier(learning_rate=0.02, seed=1234, n_estimators=n_trees, max_depth=3, min_child_weight=5, colsample_bytree=0.7, subsample=0.9)
    ]

    # Ready for cross validation
    skf = list(StratifiedKFold(y_train, n_folds))

    # Pre-allocate the data
    blend_train = np.zeros((X_train.shape[0], len(clfs))) # Number of training data x Number of classifiers
    blend_test = np.zeros((X_test.shape[0], len(clfs))) # Number of testing data x Number of classifiers

    print 'X_test.shape = %s' % (str(X_test.shape))
    print 'blend_train.shape = %s' % (str(blend_train.shape))
    print 'blend_test.shape = %s' % (str(blend_test.shape))
    
    return blend_train, blend_test, clfs

In [63]:
## Takes in classifier, training set, labels, and test set
## on which we predict stuff on

def stacking(clfs, X_train, y_train, X_test, blend_train, blend_test, y_test=None):
    
    # For each classifier, we train the number of fold times (=len(skf))
    for j, clf in enumerate(clfs):
        print 'Training classifier [%s]' % (j)
        blend_test_j = np.zeros((X_test.shape[0], len(skf))) # Number of testing data x Number of folds , we will take the mean of the predictions later
        for i, (train_index, cv_index) in enumerate(skf):
            print 'Fold [%s]' % (i)

            # This is the training and validation set
            X_dev = X_train.iloc[train_index]
            Y_dev = y_train.iloc[train_index]
            X_cv = X_train.iloc[cv_index]
            Y_cv = y_train.iloc[cv_index]

            clf.fit(X_dev, Y_dev)

            # This output will be the basis for our blended classifier to train against,
            # which is also the output of our classifiers
            blend_train[cv_index, j] = clf.predict_proba(X_cv)[:, 1]
            blend_test_j[:, i] = clf.predict_proba(X_test)[:, 1]

        # Take the mean of the predictions of the cross validation set
        blend_test[:, j] = blend_test_j.mean(1)

    print 'y_train.shape = %s' % (y_train.shape)

    # Start blending!
    bclf = LogisticRegression()
    bclf.fit(blend_train, y_train)

    # Predict now
    Y_test_predict = bclf.predict_proba(blend_test)[:, 1]
    
    if y_test:
        score = roc_auc_score(y_test, Y_test_predict)
        print 'roc_auc_score = %s' % (score)
    
    return Y_test_predict

## Correlation among predictions from different classifiers

In [64]:
## Blend Test contains mean prediction for the three classifiers across different folds
pd.DataFrame(blend_test).corr()

,0,1,2,3,4
0,1.000000,0.915909,0.681930,0.664080,0.632558
1,0.915909,1.000000,0.536156,0.525179,0.508882
2,0.681930,0.536156,1.000000,0.976112,0.954094
3,0.664080,0.525179,0.976112,1.000000,0.940204
4,0.632558,0.508882,0.954094,0.940204,1.000000


## Train on full dataset

In [70]:
blend_train, blend_test, clfs = get_blending_sets(X, y, test)

X_test.shape = (75818, 307)
blend_train.shape = (76020, 5)
blend_test.shape = (75818, 5)


In [71]:
predictions = stacking(clfs, X, y, test, blend_train, blend_test)

Training classifier [0]
Fold [0]
Fold [1]
Fold [2]
Fold [3]
Fold [4]
Training classifier [1]
Fold [0]
Fold [1]
Fold [2]
Fold [3]
Fold [4]
Training classifier [2]
Fold [0]
Fold [1]
Fold [2]
Fold [3]
Fold [4]
Training classifier [3]
Fold [0]
Fold [1]
Fold [2]
Fold [3]
Fold [4]
Training classifier [4]
Fold [0]
Fold [1]
Fold [2]
Fold [3]
Fold [4]
y_train.shape = 76020


## Submission

In [74]:
submission = pd.read_csv('../data/sample_submission.csv')
submission['TARGET'] = predictions
submission.to_csv('../submissions/stacking.csv', index=False)